# Perspective-n-Point (PnP) Navigation - pnpnav

This IPython notebook is a brief tutorial on the pnpnav package. pnpnav is a set of functionality that helps constructing a [Bundle Adjustment][bundle] problem using navigation system information. 
[bundle]:http://en.wikipedia.org/wiki/Bundle_adjustment

## PnP Overview

PnP is a subset of bundle adjustment which is a process to estimate both world structure and camera poses. PnP only tries to estimate camera pose. PnP fixes the structure of the world. We use correspondences between world and camera points as measurements. The problem is illustrated below:
![pnp_layout](img/pNp.png)

This notebook will give an overview of how the solution to this problem is computed. We will also walk through some of the code and show how it applies to the problem. In addition, we're going to outline some navigation specific *gotchas* like coordinate systems.


## PnP

As mentioned before, we are attempting to solve for camera position given the correspondences between 3D world points and their corresponding projections on the 2D image plane. This is a well studied problem, and some background on the wealth of solutions is documented in the [EP*n*P paper](http://cvlabwww.epfl.ch/~lepetit/papers/lepetit_ijcv08.pdf).

The equation for projecting a point in the local-level ENU world frame into the image is given below:

$$
  h(\mathbf{K},\mathbf{R}_w^c, \mathbf{X}^w, \mathbf{t}^w)
  = \mathbf{x}^i = \mathbf{K}\left[ \mathbf{R}_w^c\mathbf{X}^w + \mathbf{R}_w^c \mathbf{t}^w \right]
$$

This equation represents a nonlinear transformation $h()$ which uses the camera calibration matrix $\mathbf{K}$ along with the relationship of the camera to the world frame $\mathbf{R}^c_w , \mathbf{t}^w$ to project a world point $\mathbf{X}^w$ onto the image. The point on the image is denoted as $\mathbf{x}^i$. PnP tries to estimate the parameters $\mathbf{R}^c_w , \mathbf{t}^w$ in order to minimize the error between the predicted image point $\mathbf{\hat{x}}^i$ and the measured image point $\mathbf{z}^i$. Also for completeness, the point in the image plane $\mathbf{x}^i$ is normalized by its z component to end up with a vector of length 2, in pixel coordinates. 



### Coordinate Systems

A quick note on coordinate systems. In the above equation we denote each point as a vector in a Cartesian coordinate system inidicated by a superscript. For example a world point is represented by $\mathbf{X}^w$. The tranlsation of the camera in the world frame is $\mathbf{t}^w$. Rotation matrices are represented with subscript denoting the frame they are rotating from, and a superscript denoting the frame they are rotating into. In the projection equation above we have the matrix $\mathbf{R}_w^c$. This matrix rotates a vector from the world coordinate system into the camera coordinate frame. The code convention we picked @ AFIT variable names denote the superscript first, and the subscript last. So for example in the code we show a rotation matrix which rotates from an ENU frame into a camera looking straight down at the ground as:

In [1]:
import numpy as np
R_c_w = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
R_c_w

array([[ 1,  0,  0],
       [ 0, -1,  0],
       [ 0,  0, -1]])

In this problem we have three frames we need to worry about. The world frame, camera frame, and the image frame. In addition we might have some nusiance frames we need to deal with. These include the relationship between the body frame of the aircraft and the camera $R^b_c$. Usually there's a roatation between the IMU and the body frame of the aircraft $R_{imu}^b$. The truth system will output the rotation between a local level NED frame and the computation frame $R_{vehicle}^{NED}$. In UVAN we also had a rotation between the *vehicle* frame and the body frame due to some bad default settings in our code, so we had to maintain the relationship between the vehicle and body frame. $R^{b}_{vehicle}$. So you'll probably see references to these frames in the code. The diagram for the primary coordinate systems is shown below: ![pnp_frames](img/coordinate_systems.png)

### Geodetic Coordinate Systems

A couple other notes on coordinate systems we'll use. The first is WGS-84. This is a geocentric/spherical coordiante system. GPS uses this as the reference. It models the earth as an ellipsoid. Positions are defined as latitude, longitude, and height above the ellipsoid. This height is different than the geoid, which is a different model of the earth, and is the basis for Mean Sea Level (MSL). The difference between the geoid and the ellipsoid is called undulation. This difference is about 30m in most places CONUS. These differences are modeled and stored in grid shift files. Some of which are available [here](https://trac.osgeo.org/proj/wiki/VerticalDatums).

In addition, we will deal with two local level frames. The first of which we commonly refer to as the *navigation* frame. This is a Carteisan coordinate system with an origin defined at an arbitrary point in space. The frame is oriented such that it is tangent to the ellipsoid. Usually this frame is defined as **NED** or **N**orth **E**ast **D**own. Meaning that the X, Y, and Z axes point North, East, and Down, repsectively. Occasionally we deal with a rotated version of that frame such that X, Y, and Z point East, North, and Up for an **ENU** frame. This convention is usually found in 3D graphics engines. [Short reference on frames](http://www.sharpgis.net/post/2007/05/05/Spatial-references2c-coordinate-systems2c-projections2c-datums2c-ellipsoids-e28093-confusing)

The last system we're going to use is Spherical Mercator. There's a decent overview [here](http://www.maptiler.org/google-maps-coordinates-tile-bounds-projection/). This frame is used for the creation of web map tiles. It's referenced by starting with the flat earth (mercator) projection as zoom level 0. Then each zoom level is divided into quadrants. Therefore this is a quadtree based representation. Tiles are referenced by their zoom, x, and y coordinates. We're going to use this in pnpnav as a quick way to load features from a database. When creating the database we pick a fixed zoom level, and add in the x,y coordinates. Additionally we use the [Szudik Elegant Pairing Function](http://szudzik.com/ElegantPairing.pdf) to come up with a unique tile ID based on the x,y values. The features are stored in memory by this unique integer to quickly load features per tile. This project uses [mercantile](https://github.com/mapbox/mercantile) from Mapbox to manipulate/reference tiles. Here's a quick diagram with a short explination on tile systems found in web mapping utilities. 

![tile_ref](http://www.maptiler.org/img/poster-coordinates.gif)

### Camera Matrix

The last interesting matrix in the projection matrix is the *camera matrix.* The camera matrix is denoted as $\mathbf{K}$ and is computed during an *intrinsic calibration* routine. A diagram showing the main components of the camera matrix is shown below. ![cam_matrix](img/pinhole_geometry.png)

During camera calibration, we estimate the intrinsic parameters of the camera matrix. The first parameter is the **focal length** which is the distance between the camera center and the image coordinate system, denoted as $f_0$. In addition we define the prinicipal point $\mathbf{p}^i = (p_x^i, p_y^i)$. This point is defined where the camera coordinate system Z axis intersects the image plane. For computer vision applications we are interested in finding the pixel location of world points, so we use the physical size of a pixel to convert these values from meters into pixels. We denote the focal length in pixels as $\alpha = f_0 / \mu$ where $\mu$ is the pixel size in meters. The camera calibration matrix $K$ is then given as: 

$$
  \mathbf{K} = 
  \begin{bmatrix}
    \alpha   &    0             & p_x \\
        0            & \alpha   & p_y \\
        0            &    0             &  1
  \end{bmatrix}
$$

We show a camera matrix derived from one of the prosilica cameras below:

In [2]:
fm = 0.08 # meters
image_x = 1024 # pixels
image_y = 1024 # pixels
pix_size = 4.65e-06 # meters

alpha = fm / pix_size  # focal_len_pix

# Camera Matrix (pinhole)
K = np.array([[alpha, 0.0, image_x / 2],
              [0.0, alpha, image_y / 2],
              [0.0, 0.0, 1.0]])
distortion = np.zeros(5)

print(K)

[[  1.72043011e+04   0.00000000e+00   5.12000000e+02]
 [  0.00000000e+00   1.72043011e+04   5.12000000e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


### Solving PnP Problems

Now that we have most of the menial stuff out of the way, let's figure out where we're at. Again, pnp works by trying to minimize the error between the predicted projection locations in the image frame with where they actually show up. Ignoring feature matching problems for the moment we set up an indiviual residual value as the difference between the predcited feature location for the $k^{th}$ feature $\hat{\mathbf{x}}^i_k$ and its measured location $\mathbf{z}_k$. Using current state estimate of where the camera is located in the local level ENU frame $\mathbf{t}^w$ and the rotation from ENU to the Camera coordinate system $\mathbf{R}^c_w$. Using the projection equation above, we define this $k^{th}$ residual value as:

$$
r_k = \mathbf{z}_k - \mathbf{K}\left[ \mathbf{R}_w^c\mathbf{X}^w + \mathbf{R}_w^c \mathbf{t}^w \right]
$$

We define the total cost function as the squared L2 norm of a vector consisting of the $n$ residuals computed for each observed feature-world point pair:

$$
\newcommand{\norm}[1]{\|{#1}\|}
\mathbf{F}(\mathbf{t}^w, \mathbf{R}_w^c) = \norm{ ( r_0 , \dots , r_n )}_2^2
$$

We try to minimize this non-linear cost function with pretty standard techniques. The [authors of Google Ceres](http://ceres-solver.org/nnls_solving.html#chapter-nnls-solving) do a much better job of writing up solving non-linear least squares problems than I can. Basically at each step we look at the parital deriviates of $F(x)$ with respect to the different parameters $\mathbf{R}^c_w, \mathbf{t}^w$ to figure how to adjust the parameters to decrease $F$. Levenberg-Marquardt is an iterative trust-region method, where we look at solving a series of linearized versions of $F$. We keep iterating until the problem converges. The most difficult part is formulating an update process at each step to maintain the internal constraints within the rotation matrix parameters. 

OpenCV has a method that given a camera calibration matrix, ENU coordinates of world points, and the measured 2D points, will output the rotation matrix and translation vector. You can specify different solvers (e.g. Levenberg Marquardt, EPnP). You can also have it perform RANSAC, which is a method to try to remove outliers by finding random pairs of points that meet a consensus for the answer. The core.py in pnpnav implements helper functions to get 2D/3D correspondences into the right format for OpenCV to use. Additonally helps turn that output back into something more usable for navigation (e.g. WGS-84 coordinates). 


## PnP Code Example

Lets get into solving a problem with pnpnav. We're going to investigate the code I wrote for the unit tests and basically replicate the core unit test. First lets import the helper functions:

In [3]:
from pnpnav._shared.testing import *

First let's get some ground points defined. To do this, we define a point on the earth. I picked a Lat and Lon from WPAFB. We then find the spherical mercator tile containing this point, at a specified zoom level. The `generate_ground_points()` function in `pnpnav._shared.testing` will implement the generation of ground points. We'll step thru that here. First, we pick a zoom level of 15 (~1.2km square) and find the correct tile

In [4]:
zoom = 15
    wpafb_lon = -84.049954
    wpafb_lat = 39.8179055
    tile = mercantile.tile(wpafb_lon, wpafb_lat, zoom)
    print(tile)

Tile(x=8733, y=12426, z=15)


After we find the tile, we generate `num_points` uniformly distributed latitude and longitude points. We use mercantile to calculate the bounds of the tile, and use those to figure out the parameters to pass into `np.random.uniform`. We then use a Gaussian distribution for the heights, with a $\sigma$ value of `10.0`

In [5]:
vert_sigma = 10.0
    num_pts = 1000.0
    bounds = mercantile.bounds(tile.x, tile.y, tile.z)
    lons = np.random.uniform(bounds.west, bounds.east, num_pts)
    lats = np.random.uniform(bounds.south, bounds.north, num_pts)
    heights = np.random.standard_normal(num_pts) * vert_sigma
    center = np.array([(bounds.west + bounds.east) / 2,
                       (bounds.north + bounds.south) / 2,
                       0.0])

Finally we use some of the tools from navpy to also calculate the NED coordinates

In [6]:
pts_ned = navpy.lla2ned(lats, lons, heights, center[1], center[0], 0.0)
    lon_lat_height = np.vstack((lons, lats, heights)).T

Next, we want to define our camera parameters. The testing suite has a built in function `generate_cam_data()` that builds a camera model. We'll pull that code out here for clarity, but it's the same example code we used before. 

In [7]:
image_x = image_y = 1024
fm=0.08
pix_size=4.65e-06
alpha = fm / pix_size  # focal_len_pix

# Camera Matrix (pinhole)
K = np.array([[alpha, 0.0, image_x / 2],
              [0.0, alpha, image_y / 2],
              [0.0, 0.0, 1.0]])
distortion = np.zeros(5)
print(K)

[[  1.72043011e+04   0.00000000e+00   5.12000000e+02]
 [  0.00000000e+00   1.72043011e+04   5.12000000e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


Next we want to find the optimal altitude for imaging the entire tile. To do so, we use the camera parameters to figure out the field of view of the camera, and we set the altitude to let the ground field of view be the size of the tile. This is implemented in `find_optimal_altitude()`

In [8]:
mean_gp = (pts_ned[:, 0:2]).mean(0)
mg = np.abs(pts_ned[:, 0:2] - mean_gp).max()
angle = np.arctan(image_x / 2.0 / K[0, 0])
alt = np.ceil(mg / np.tan(angle))

# Go to the nearest 10m multiple just to be safe
alt = alt + (10 - np.mod(alt, 10))
print("Optimal Altitude to View Tile: %f m" % alt)

Optimal Altitude to View Tile: 15920.000000 m


Now we need to figure out the camera pose. We start with defining $R_{nadir}$ which is the basis for $R^{cam}_{ENU}$ that points the camera straight down toward the ENU plane. Notice that the X axes are aligned, with the Y and Z reversed.

In [9]:
# Generate a nominal camera pose
R_nadir = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
print(R_nadir)

[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]


If we wish, we can perturb this very slightly by setting `att_sigma` $ > 0$. However for this test case we'll leave it to be a pure nadir view of the ground. In addition, we'll leave the camera centered over the tile.

In [13]:
att_sigma = 0.0
roll = att_sigma * np.random.standard_normal() * (np.pi / 180.0)
pitch = att_sigma * np.random.standard_normal() * (np.pi / 180.0)
yaw = att_sigma * np.random.standard_normal() * (np.pi / 180.0)
R_c_nadir = navpy.angle2dcm(roll, pitch, yaw, rotation_sequence='ZYX')
R_c_w = np.dot(R_c_nadir, R_nadir).T

# t_w - translation between world and camera frame center,
# expressed in world coordinates
east_m = 0.0
north_m = 0.0
t_w = np.array([east_m, north_m, alt])
print("ENU to world Rotation Matrix: \n%s" % R_c_w)
print("Camera in ENU coordinates: %s" % t_w)

ENU to world Rotation Matrix: 
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Camera in ENU coordinates: [     0.      0.  15920.]


Now, we have ground points, $\mathbf{X}^{NED}$, so let's quickly convert them to ENU.

In [14]:
gp_enu = np.hstack((pts_ned[:, [1, 0]], -1 * pts_ned[:, [2]]))

Now we have in the workspace:
* World Points - `gp_enu` - $\mathbf{X}^{ENU}$
* Camera Calibration Matrix - `K` - $\mathbf{K}$
* Camera Position in ENU Frame - `t_w` - $\mathbf{t}^{ENU}$
* Rotation Matrix from ENU to Camera Frame - `R_c_w` - $\mathbf{R}^c_{ENU}$

With all this we should be ready to calculate the positions of the world points in our image frame using the projection equation. We do a little bit of transpoing in order to avoid for loops in the code.

First we rotate the ground points into the camera frame

In [15]:
x0 = np.dot(R_c_w, gp_enu.T)

Then we find the translation of the points needed to account for the offset of the camera center point.

In [16]:
tc = np.dot(R_c_w, t_w).T
x_cam = (x0.T - tc).T

Now we have the points in the camera coordinate system, `x_cam`:

$$\mathbf{X}^{cam} = \mathbf{R}^{cam}_{ENU}\mathbf{X}^{ENU} - \mathbf{R}^{cam}_{ENU}\mathbf{t}^{ENU}$$

And we calculate the unnormalized 3D points in the image frame using the camera projection matrix $\mathbf{K}$. We then have our final pixel coordinates by normalizing by the z values. Printing the max and min values shows that we have pixel values within our camera spec (`1024x1024`)

In [17]:
x_img = np.dot(K, x_cam)
x_img = (x_img[0:2, :] / x_img[2]).T
print (x_img.max(), x_img.min())

(1019.2282216455931, 4.1477524376051047)


I wrote an input function for PnP that takes these corresponding numpy arrays and wraps them in a class to pass to PnP, called `matching.FeatureCorrespondence2D3D()`. This would be the ideal output of one of the matchers. Let's call it on our 2D/3D matches and see what the output looks like. 

In [18]:
matches = build_matches(lon_lat_height, x_img)
matches.keypoints

array([[ 122.7818037 ,  200.09312848],
       [ 860.35618388,  999.51392482],
       [ 856.93767606,  868.0404896 ],
       ..., 
       [ 304.35230459,  866.78397531],
       [ 539.81747387,  312.58792609],
       [ 305.18393476,  917.18050566]])

`matches.keypoints` is simply the `Nx2 numpy.ndarray` of the image (x,y) points. 

In [19]:
matches.world_coordinates

array([[-84.05511168,  39.82379441,  -5.57970519],
       [-84.04713369,  39.81712587, -20.10646623],
       [-84.04717673,  39.81822784,   5.87500202],
       ..., 
       [-84.0531462 ,  39.81823897,   9.2390871 ],
       [-84.05060249,  39.82285695,  -8.87174988],
       [-84.05313604,  39.81782097,  17.55116527]])

`matches.world_coordinates` is the `Nx3 numpy.ndarray` of the World Points in Longitude (deg), Latitude (deg), and Height Above Ellipsoid (m). And we'll just check to see if we have `1000` correspondences:

In [20]:
matches.num_correspondences

1000

Now we are finally ready to try to figure out where we're at! I'm going to rip out the internals of `PnP.__opencv_pnp(self, new_matches)`. First we'll walk thru some of the other functions in PnP just so that we know what's going on.

* `use_fundamental_matrix_constraint()` tells the PnP object to use the fundamental matrix constraint to try to find outliers before calling the actual PnP routine
* `use_homography_constraint()` does the same thing except uses the homography matrix. This is better when you dont have a lot of terrain relief. 
* `__apply_geometric_constraint()` is a "private" function that gets called inside the PnP routine to apply either the F or H matrix constraint
* `load_camera_parameters` loads an OpenCV formatted .yaml file that has the camera parameters into the PnP member variables
* `load_pytables_db()` is a function that tells `PnP.__matcher` to load a specific database file
* `set_db_location_from_tile(self, tile)` is another passthru that tells matcher to load features from the database into the matcher memory and search structure
* `do_pnp(self, query_kp, query_desc)` is a two part function. It's abstracted so that whatever implementation you end up using just has to know to call do_pnp. You pass in the `query_kp` which is an `Mx2` sized array of `(x,y)` feature points. `query_desc` is an `Mx128` sized vector of un-nomralized SIFT feature descriptor vectors. `do_pnp` passes these to the internal `__matcher` object to do the image-space matching (using FLANN or brute force). Then it calls whatever function to actually do pnp. In this case it calls `__opencv_pnp(self, new_matches)`
* Finally we're going to step thru `__opencv_pnp(self, new_matches)`. This function takes in new_matches, which is of type ``matching.FeatureCorrespondence2D3D()`. We have already created this object in this tutorial, so we'll use that. We just need to be a little tricky and set up the other internal member objects. Let's first declare a pnp object:

In [21]:
import pnpnav.core as core
pnp = core.PnP()

Now we need to get fancy and overload some of the internal member variables of `pnp` with the values we've created in the tutorial. Python garbles any variables you have declared/intended to be private with two leading underscores such that it'll look a little weird here:


In [22]:
pnp._PnP__camera_matrix = K
pnp._PnP__distortion = distortion

Also, I'm going to be lazy and overload some of the names we've come up with so that I don't have to rewrite the code inside of `__opencv_pnp(self, new_matches)`.

In [23]:
self = pnp
new_matches = matches
keypoints = x_img
lon_lat_h = lon_lat_height

It will seem like we're going slightly backwards here, but stay with me. We dont know anything about our local level coord frame we created earlier. So in this case we are going to create a new one using the first point in `new_matches`.

In [24]:
ref = lon_lat_h[0, :]
world_pts_ned = navpy.lla2ned(
    lon_lat_h[:, 1], lon_lat_h[:, 0], lon_lat_h[:, 2],
    ref[1], ref[0], ref[2])
print("Reference Lon, Lat, H: %s" % ref)

Reference Lon, Lat, H: [-84.05511168  39.82379441  -5.57970519]


So now we have the world points in an NED frame, centered at ref. We aren't worried about outliers at this point, so we don't need to apply the F or H matrix. We'll still call it to get the `idx` vector the rest of the code needs.

In [25]:
idx = self._PnP__apply_geometric_constraint(
    world_pts_ned[:, 0:2].astype(np.float32),
    keypoints.astype(np.float32))
print("%d kp from Matcher :: %d passed geometric constraint " %
      (new_matches.num_correspondences, idx.shape[0]))

1000 kp from Matcher :: 1000 passed geometric constraint 


Now we have everything setup, we can actually call the OpenCV `solvePnPRansac` function. We pass in the world points in NED, the 2D image keypoints, the camera matrix, our distortion parameters, and the RANSAC threshold for outlier rejection. This function returns a Rodrigues rotation vector from local level to camera, and the translation vector of the origin of the local level frame in camera coordinates.

In [26]:
import cv2
rvec, tvec, pnp_status = \
    cv2.solvePnPRansac(
        world_pts_ned[idx, :].astype(np.float32),
        keypoints[idx, :].astype(np.float32),
        self._PnP__camera_matrix,
        self._PnP__distortion,
        reprojectionError=1.0)

In [27]:
print("Rotation Vec: %s" % rvec.flatten())
print("Translation Vec: %s" % tvec.flatten())

Rotation Vec: [ -7.99550771e-05   8.65403181e-06  -1.57074929e+00]
Translation Vec: [  -360.28966665   -288.72448522  15925.59642646]


Now we need to just rotate/transform these vectors into the direction cosine matrices and coordinate systems we're familiar with:

In [28]:
# Rvec is a rodrigues vector from world to cam, so need to
# transpose

C_n_b = (cv2.Rodrigues(rvec)[0]).transpose()

# Then tvec is from world to cam, so need to rotate into
# world frame and negate

t_nav = -1 * np.dot(C_n_b, tvec.reshape(3, 1)).flatten()
pnp_wgs = navpy.ned2lla(t_nav, ref[1], ref[0], ref[2])

Finally we convert our truth position to LLH:

In [29]:
t_ned = np.hstack((t_w[[1, 0]], -1 * t_w[2]))
t_wgs = navpy.ned2lla(t_ned, center[1], center[0], center[2])

In [30]:
print("True WGS: %s " % np.array(t_wgs))
print("Calc WGS: %s " % np.array(pnp_wgs))
print("Delta: %s" % (np.array(t_wgs) - np.array(pnp_wgs)))

True WGS: [    39.8211941     -84.05090332  15920.0000027 ] 
Calc WGS: [    39.8211941     -84.05090332  15920.00000805] 
Delta: [  1.12993348e-09   2.03613126e-10  -5.35044819e-06]


There we go. Now you know where you're at. Next steps are figuring out the implementation of the pytables feature database, and the matching algorithms. But that's a trivial exercise left to the reader :)